In [16]:
!pip install pymdptoolbox

In [17]:
import numpy as np
import math
import matplotlib.pyplot as plt
import itertools as it
import mdptoolbox
import pandas as pd

In [18]:

train_data = pd.read_csv('warehousetraining.txt', sep='\t')
train_data.columns = ["Actions", "Storage State"]

test_data = pd.read_csv('warehouseorder.txt', sep='\t')
test_data.columns = ["Actions", "Storage State"]

print("\nTraining Data:\n", train_data)
print("\nTesting Data:\n", test_data)
print("\nCount of Training Data:", len(train_data))
print("\nCount of Testing Data:", len(test_data))


Training Data:
        Actions Storage State
0        store          blue
1        store         white
2      restore          blue
3      restore         white
4        store         white
...        ...           ...
12102  restore           red
12103  restore          blue
12104  restore           red
12105  restore           red
12106  restore           red

[12107 rows x 2 columns]

Testing Data:
     Actions Storage State
0     store          blue
1     store         white
2   restore          blue
3   restore         white
4     store         white
5     store           red
6     store         white
7     store           red
8   restore           red
9   restore           red
10    store         white
11    store          blue
12    store         white
13  restore           red
14  restore         white
15  restore         white
16    store           red
17    store          blue
18  restore           red
19    store          blue
20    store         white
21  restore         w

In [19]:
def get_states():
    storage_states = ['b','r','w','e']
    actions = ['sto_r', 'sto_b', 'sto_w', 'res_r', 'res_b', 'res_w']
    states = list(it.product(storage_states,storage_states,storage_states,storage_states,actions))               
    return states

states = get_states()
print(len(states))
print(states)

1536
[('b', 'b', 'b', 'b', 'sto_r'), ('b', 'b', 'b', 'b', 'sto_b'), ('b', 'b', 'b', 'b', 'sto_w'), ('b', 'b', 'b', 'b', 'res_r'), ('b', 'b', 'b', 'b', 'res_b'), ('b', 'b', 'b', 'b', 'res_w'), ('b', 'b', 'b', 'r', 'sto_r'), ('b', 'b', 'b', 'r', 'sto_b'), ('b', 'b', 'b', 'r', 'sto_w'), ('b', 'b', 'b', 'r', 'res_r'), ('b', 'b', 'b', 'r', 'res_b'), ('b', 'b', 'b', 'r', 'res_w'), ('b', 'b', 'b', 'w', 'sto_r'), ('b', 'b', 'b', 'w', 'sto_b'), ('b', 'b', 'b', 'w', 'sto_w'), ('b', 'b', 'b', 'w', 'res_r'), ('b', 'b', 'b', 'w', 'res_b'), ('b', 'b', 'b', 'w', 'res_w'), ('b', 'b', 'b', 'e', 'sto_r'), ('b', 'b', 'b', 'e', 'sto_b'), ('b', 'b', 'b', 'e', 'sto_w'), ('b', 'b', 'b', 'e', 'res_r'), ('b', 'b', 'b', 'e', 'res_b'), ('b', 'b', 'b', 'e', 'res_w'), ('b', 'b', 'r', 'b', 'sto_r'), ('b', 'b', 'r', 'b', 'sto_b'), ('b', 'b', 'r', 'b', 'sto_w'), ('b', 'b', 'r', 'b', 'res_r'), ('b', 'b', 'r', 'b', 'res_b'), ('b', 'b', 'r', 'b', 'res_w'), ('b', 'b', 'r', 'r', 'sto_r'), ('b', 'b', 'r', 'r', 'sto_b'), ('

In [20]:
def get_tpm(storage_size):
    
    #no = number of...
    storage_states = ['b','r','w','e']
    actions = ['sto_r', 'sto_b', 'sto_w', 'res_r', 'res_b', 'res_w']
    no_actions = len(actions)
    s_size = storage_size
    no_items = len(storage_states)

    total_items = pow(storage_size, no_items)
    all_states = total_items * no_actions

    tp = []

    for i in range(storage_size):
        tp.append(np.zeros((all_states,all_states), dtype=np.float16))

    probabilities = {'sto_r':0.2469, 'sto_b':0.1253, 'sto_w':0.1278, 'res_r':0.2469, 'res_b':0.1253, 'res_w':0.1278}

    for i ,transition in enumerate(tp):
        for j, state in enumerate(states):
            for k, next_state in enumerate(states):
                
                # storing
                if state[-1][0] != 'b' or 'r' or 'w':
                    
                    # storing is possible
                    if state[i] == '0':
                        curr_state_store = state[:i] + (state[-1][1],) + state[i + 1:-1]
                        if curr_state_store == next_state[:-1]:
                            transition[j][k] = probabilities[next_state[-1]]
                        continue
                    # storing is not possible
                    if state[:-1] == next_state[:-1]:
                        transition[j][k] = probabilities[next_state[-1]]
                        continue
                        
                # restoring 
                if state[-1][0] == 'r':
                    
                    if state[-1][1] == 'r':
                        # restoring possible
                        if state[i] == 'r':
                            curr_state_restore = state[:i] + ('0',) + state[i + 1:-1]
                            if curr_state_restore == next_state[:-1]:
                                transition[j][k] = probabilities[next_state[-1]]
                            continue
                        # restoring not possible
                        if state[:-1] == next_state[:-1]:
                            transition[j][k] = probabilities[next_state[-1]]
                            continue
                    
                    if state[-1][1] == 'w':
                        # restore possible
                        if state[i] == 'w':
                            if curr_state_restore == next_state[:-1]:
                                transition[j][k] = probabilities[next_state[-1]]
                            continue
                        # restoring not possible
                        if state[:-1] == next_state[:-1]:
                            transition[j][k] = probabilities[next_state[-1]]
                            continue
                    
                    if state[-1][1] == 'b':
                        # restore possible
                        if state[i] == 'b':
                            if curr_state_restore == next_state[:-1]:
                                transition[j][k] = probabilities[next_state[-1]]
                            continue
                        # restoring not possible
                        if state[:-1] == next_state[:-1]:
                            transition[j][k] = probabilities[next_state[-1]]
                            continue
    
    return tp

tp_arr = np.array(tp)
tp_arr= get_tpm(4)
print(tp_arr)
print(np.shape(tp_arr))


[array([[0.247 , 0.1252, 0.1278, ..., 0.    , 0.    , 0.    ],
       [0.247 , 0.1252, 0.1278, ..., 0.    , 0.    , 0.    ],
       [0.247 , 0.1252, 0.1278, ..., 0.    , 0.    , 0.    ],
       ...,
       [0.    , 0.    , 0.    , ..., 0.247 , 0.1252, 0.1278],
       [0.    , 0.    , 0.    , ..., 0.247 , 0.1252, 0.1278],
       [0.    , 0.    , 0.    , ..., 0.247 , 0.1252, 0.1278]],
      dtype=float16), array([[0.247 , 0.1252, 0.1278, ..., 0.    , 0.    , 0.    ],
       [0.247 , 0.1252, 0.1278, ..., 0.    , 0.    , 0.    ],
       [0.247 , 0.1252, 0.1278, ..., 0.    , 0.    , 0.    ],
       ...,
       [0.    , 0.    , 0.    , ..., 0.247 , 0.1252, 0.1278],
       [0.    , 0.    , 0.    , ..., 0.247 , 0.1252, 0.1278],
       [0.    , 0.    , 0.    , ..., 0.247 , 0.1252, 0.1278]],
      dtype=float16), array([[0.247 , 0.1252, 0.1278, ..., 0.    , 0.    , 0.    ],
       [0.247 , 0.1252, 0.1278, ..., 0.    , 0.    , 0.    ],
       [0.247 , 0.1252, 0.1278, ..., 0.    , 0.    , 0.    ],

In [21]:
def get_rewards(storage_size):
    
    storage_states = ['b','r','w','e']
    actions = ['sto_r', 'sto_b', 'sto_w', 'res_r', 'res_b', 'res_w']
    no_actions = len(actions)
    s_size = storage_size
    no_items = len(storage_states)
    total_items = pow(storage_size, no_items)
    all_states = total_items * no_actions

    rm = np.zeros((all_states,storage_size))
    
    reward = [-1,-2,-2,-3,-3,-4]
    
    for i, state in enumerate(states):
        
        if state[-1][0] == 'e':
         #if state[-1][0] != 'b' or 'r' or 'w':
            for j in range(storage_size):
                rm[i][j] = reward[j] if state[j] == '0' else -10
        
        else:
            for j in range(storage_size):
                rm[i][j] = reward[j] if state[j] == state[-1][1] else -100

    return rm

rm = get_rewards(4)
print(rm)
print(rm.shape)

[[-100. -100. -100. -100.]
 [-100. -100. -100. -100.]
 [-100. -100. -100. -100.]
 ...
 [  -1.   -2.   -2.   -3.]
 [  -1.   -2.   -2.   -3.]
 [  -1.   -2.   -2.   -3.]]
(1536, 4)


In [35]:
mdpresultPolicy =  mdptoolbox.mdp.FiniteHorizon(tp_arr, rm, 0.99, N =5)
mdpresultPolicy.run()

print('Policy Iteration Results:')
print('\nPolicy Values:\n',mdpresultPolicy.policy)
print('\nValue Function values:\n',mdpresultPolicy.V)

Policy Iteration Results:

Policy Values:
 [[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 ...
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]

Value Function values:
 [[-490.099501   -394.0399     -297.01       -199.         -100.
     0.        ]
 [-490.099501   -394.0399     -297.01       -199.         -100.
     0.        ]
 [-490.099501   -394.0399     -297.01       -199.         -100.
     0.        ]
 ...
 [-198.00024801 -149.4901495  -100.49005     -50.995        -1.
     0.        ]
 [-198.00024801 -149.4901495  -100.49005     -50.995        -1.
     0.        ]
 [-198.00024801 -149.4901495  -100.49005     -50.995        -1.
     0.        ]]


In [37]:
mdpresultPolicy =  mdptoolbox.mdp.PolicyIteration(tp_arr, rm, 0.99, max_iter = 100000)
mdpresultPolicy.run()

print('Policy Iteration Results:')
print('\nPolicy Values:\n',mdpresultPolicy.policy)
print('\nValue Function values:\n',mdpresultPolicy.V)
print('\nNo of Iterations:\n',mdpresultPolicy.iter)

Policy Iteration Results:

Policy Values:
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0,

In [23]:
mdpresultValue = mdptoolbox.mdp.PolicyIterationModified(tp_arr, rm, 0.99, epsilon=0.01, max_iter=10)
mdpresultValue.run()
print('Policy Iteration Modified:')
print('\nPolicy Values:\n',mdpresultPolicy.policy)
print('\nValue Function values:\n',mdpresultPolicy.V)
print('\nNo of Iterations:\n',mdpresultPolicy.iter)

Policy Iteration Modified:

Policy Values:
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0

In [24]:
mdpresultValue = mdptoolbox.mdp.QLearning(tp_arr,rm, 0.99, n_iter=10000)
mdpresultValue.run()
print('QLearning:')
print('\nPolicy Values:\n',mdpresultPolicy.policy)
print('\nValue Function values:\n',mdpresultPolicy.V)
print('\nNo of Iterations:\n',mdpresultPolicy.iter)

QLearning:

Policy Values:
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1,

In [25]:
mdpresultValue = mdptoolbox.mdp.RelativeValueIteration(tp_arr, rm, 0.99, max_iter=100)
mdpresultValue.run()
print('RelativeValueIteration:')
print('\nPolicy Values:\n',mdpresultPolicy.policy)
print('\nValue Function values:\n',mdpresultPolicy.V)
print('\nNo of Iterations:\n',mdpresultPolicy.iter)

RelativeValueIteration:

Policy Values:
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0

In [26]:
mdpresultValue = mdptoolbox.mdp.ValueIteration(tp_arr, rm, 0.99, max_iter=100)
mdpresultValue.run()
print('ValueIteration:')
print('\nPolicy Values:\n',mdpresultPolicy.policy)
print('\nValue Function values:\n',mdpresultPolicy.V)
print('\nNo of Iterations:\n',mdpresultPolicy.iter)

ValueIteration:

Policy Values:
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 

In [27]:
mdpresultValue = mdptoolbox.mdp.ValueIterationGS(tp_arr, rm, 0.99, max_iter=10)
mdpresultValue.run()
print('ValueIterationGS:')
print('\nPolicy Values:\n',mdpresultPolicy.policy)
print('\nValue Function values:\n',mdpresultPolicy.V)
print('\nNo of Iterations:\n',mdpresultPolicy.iter)

ValueIterationGS:

Policy Values:
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1